In [ ]:
# dataframe management
import pandas as pd             

# numerical computation
import numpy as np

import re

# visualization library
import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")

from bokeh.layouts import gridplot
from bokeh.plotting import figure

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()

from datetime import datetime, timedelta, date
from scipy.stats import skew
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.datasets import load_boston

import statsmodels.api as sm


# Functions definitions

In [ ]:
def toDate(x):
    parts = [int(el) for el in x.split("/")]
    return date(parts[2], parts[1], parts[0])

def r2_cv(model, sales_train, y, random_state=12345678):
    r2= cross_val_score(model, sales_train, y, scoring="r2", cv =KFold(10, shuffle=True, random_state=random_state)) 
    return(r2)

def rmse_cv(model, sales_train, y, random_state=12345678):
    rmse= np.sqrt(-cross_val_score(model, sales_train, y, scoring="neg_mean_squared_error", cv =KFold(10, shuffle=True, random_state=random_state)))
    return(rmse)

def RegionError(region, data):
    d = data[data[region] == 1][["StoreID","Month","NumberOfSales","NumberOfPredictedSales"]].groupby(["StoreID","Month"]).agg("sum")
    res = abs(d["NumberOfSales"]-d["NumberOfPredictedSales"]).agg("sum")
    return res / d["NumberOfSales"].agg("sum")

def q(col, quant, f):
    t = sales[col].quantile(quant)
    print(f'col {col} at {quant}-th quantile => {t}')
    sales.loc[f(sales[col], t), col] = t
    
def getFilterRegion(cluster, data):
    filterRegion = data[cluster[0]] == 1
    for region in cluster[1:]:
        filterRegion = ((filterRegion) | (data[region]==1))
    return filterRegion

def getColsMatching(data, oldCol):
    return [col for col in data.columns if re.match(r"("+oldCol+"_)(\d)", col)]

def dedummify(data, oldCol):
    return data[getColsMatching(data,oldCol)].idxmax(axis=1).apply(lambda x : x.split("_")[-1])

def Fit(cluster, train, test, model_simple_step2):
    
    #Get only the data for the required cluster in train and test set
    train_region_label= train.loc[getFilterRegion(cluster,train)]
    test_region_label= test.loc[getFilterRegion(cluster,test)]
    
    #removing the region columns in order to force the tree alghoritm to do not split for regions
    cols_not_for_step1 = ['Region_PopulationK','Region_AreaKM2','Region_GDP',*cluster]
    train_region_columns = train_region_label[cols_not_for_step1]
    train_region_label=train_region_label.drop(columns=cols_not_for_step1)

    test_region_columns = test_region_label[cols_not_for_step1]
    test_region_label=test_region_label.drop(columns=cols_not_for_step1)
    
    train_x_region_label_step1 = train_region_label.drop(columns=['NumberOfSales','NumberOfCustomers'])
    test_x_region_label_step1 = test_region_label.drop(columns=['NumberOfSales','NumberOfCustomers'])
    train_y_region_label_step1 = pd.DataFrame(data = train_region_label['NumberOfCustomers'])
    test_y_region_label_step1 = pd.DataFrame(data = test_region_label['NumberOfCustomers']) 
    
    #First model predicting NumberOfCustomers
    model_simple = RandomForestRegressor()
    model_simple = model_simple.fit(train_x_region_label_step1, train_y_region_label_step1)
    yp = model_simple.predict(test_x_region_label_step1) #yp=predicted customers del test
    
    test_region_label[cols_not_for_step1] = test_region_columns[cols_not_for_step1]
    train_region_label[cols_not_for_step1] = train_region_columns[cols_not_for_step1]
    
    # Prepare data for step2
    train_x_step2 = train.drop(columns=['NumberOfSales'])
    train_y_step2 = pd.DataFrame(data = train['NumberOfSales'])
    test_x_region_label_step2 = test_region_label.drop(columns=['NumberOfSales','NumberOfCustomers'])
    test_x_region_label_step2['NumberOfCustomers']= yp
    test_y_region_label_step2 = pd.DataFrame(data = test_region_label['NumberOfSales']) 
    
    cols_for_step2 = [*getColsMatching(train,"Month"), *getColsMatching(train,"Region"),'NumberOfCustomers','Region_AreaKM2','HasPromotions','IsHoliday','Region_GDP', "StoreID"]
    train_x_step2 = train_x_step2[cols_for_step2]
    test_x_region_label_step2 = test_x_region_label_step2[cols_for_step2]
    
    #STEP2
    if model_simple_step2 == None:
        model_simple_step2 = RandomForestRegressor()
        model_simple_step2 = model_simple_step2.fit(train_x_step2, train_y_step2)
    yp2 = model_simple_step2.predict(test_x_region_label_step2) #yp2 = le sales predette alla fine
    
    # Restore Month columns after being dummified to ease the error estimation
    test_x_region_label_step2["Month"]=dedummify(test_x_region_label_step2, r"Month")
    # Prepare needed data
    check = pd.DataFrame(test_y_region_label_step2)
    check["NumberOfPredictedSales"] = yp2
    check["StoreID"] = test_x_region_label_step2[["StoreID"]]
    check['Month'] = test_x_region_label_step2['Month']
    check[cluster] = test_region_columns[cluster]
    
    errs = []
    for region_label in cluster:
        considered_region = (test_x_region_label_step2[region_label]==1).tolist()
        check2 = check[considered_region]
        check2[region_label]=1
        errs.append(RegionError(region_label, check2))
        print("Region " + (region_label.split("_")[-1]) + " : ", errs[-1])
    return model_simple_step2, errs

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like wstepwise_selectionith the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame( X.astype(float)[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame( X.astype(float)[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# Read input data

In [ ]:
# Reading the data
sales = sales_string_date=pd.read_csv('train.csv')

# Removing tuples where stores are closed
sales = sales[sales['IsOpen'] == 1]

# Converting to category columns that are labels
for el in ["StoreID", "Region"]:
    sales[el] = sales[el].astype("category")

In [ ]:
nulls = sales.isnull().sum()
sorted([(x,y) for (x,y) in zip(nulls.index, nulls) if y>0], key=lambda x: x[1], reverse=True)

We see that Max_Gust_SpeedKm_h has 409947 missing values. We decided not to impute it.

Let's start with imputation of "Events"

In [ ]:
null_Events = sales['Events'].isnull()
event_missing = sales[null_Events]
event_missing.describe()

In [ ]:
sales['Events'].value_counts()

By this, we discovered that when Event is null the weather is good, from the fact that Precipitationmm mean is almost 0.
Furthermore, all the labels of Events are related to bad weather, that means that when no precipitation occurs the label is null.
We will impute Event by replacing the missing values with "Not Specified" (later).

Now we impute "CloudCover", making a distinction when it misses along with Events and when it misses on its own.

In [ ]:
null_Events = sales['Events'].isnull()
null_CloudCover = sales['CloudCover'].isnull()
cloudcover_missing = sales[(null_Events)]
null_Events = sales['Events'].isnull()
event_missing = sales[null_Events]
event_cc_missing = sales[null_CloudCover & null_Events]

cloudcover_missing.shape, event_missing.shape, event_cc_missing.shape 

There are 28k tuples where both "Events" and "CloudCover" are missing, that means that the weather should be good.
for the remaining (41k-28k) (i.e. where "Events" is not null!) tuples we impute the CloudCoverage.

In [ ]:
event_notmissing_cc_missing = sales[null_CloudCover & ~null_Events]
#computing mean where CloudCover is not null
mean_CC = sales["CloudCover"].mean()
event_notmissing_cc_missing["CloudCover"] = event_notmissing_cc_missing["CloudCover"].fillna(mean_CC)
sales = pd.concat([sales[~null_CloudCover | null_Events], event_notmissing_cc_missing])

Then, we impute the remaining rows with CloudCover missing

In [ ]:
null_Events = sales['Events'].isnull()
CloudyButNotEvent = sales[null_Events]
#there are some tuples with no Events but with the attribute CloudCover

In [ ]:
null_cloudCover = sales[sales['CloudCover'].isnull()]
mean_cloudCover = sales["CloudCover"].mean()
null_cloudCover["CloudCover"] = null_cloudCover["CloudCover"].fillna(mean_cloudCover)

sales = pd.concat([sales[~sales['CloudCover'].isnull()], null_cloudCover])

Now, we impute min,max,mean_VisibilityKm. 

In [ ]:
# checking if are all the same rows where visibility data are missing => yes, they are
sales[["Max_VisibilityKm", "Mean_VisibilityKm", "Min_VisibilitykM"]].count()

In [ ]:
null_visibility = sales[sales['Max_VisibilityKm'].isnull()]
mean_vis_max = sales["Max_VisibilityKm"].mean()
mean_vis_mean = sales["Mean_VisibilityKm"].mean()
mean_vis_min = sales["Min_VisibilitykM"].mean()

null_visibility["Max_VisibilityKm"] = null_visibility["Max_VisibilityKm"].fillna(mean_vis_max)
null_visibility["Mean_VisibilityKm"] = null_visibility["Mean_VisibilityKm"].fillna(mean_vis_mean)
null_visibility["Min_VisibilitykM"] = null_visibility["Min_VisibilitykM"].fillna(mean_vis_min)

sales = pd.concat([sales[~sales['Max_VisibilityKm'].isnull()], null_visibility])

Finally, we impute "Events".

In [ ]:
sales=sales.replace(np.nan,'NotSpecified', regex=True)


# Dealing with outliers

In [ ]:
sales.quantile(.99).sort_values(ascending=False).head(20)

In [ ]:
q("NearestCompetitor", .95, lambda x, y: x > y)
q("Precipitationmm", .95, lambda x, y: x > y)
q("Max_Wind_SpeedKm_h", .95, lambda x,y: x > y)
q("Max_Wind_SpeedKm_h", .03, lambda x,y: x < y)
q("Max_TemperatureC", .95, lambda x,y: x > y)
q("Max_TemperatureC", .03, lambda x,y: x < y)
q("Min_TemperatureC", .95, lambda x,y: x > y)
q("Min_TemperatureC", .03, lambda x,y: x < y)
q("Mean_Dew_PointC", .95, lambda x,y: x > y)
q("Mean_Dew_PointC", .05, lambda x,y: x < y)
q("Mean_Dew_PointC", .95, lambda x,y: x > y)
q("Mean_Dew_PointC", .05, lambda x,y: x < y)
q("Mean_Humidity", .95, lambda x,y: x > y)
q("Mean_Humidity", .03, lambda x,y: x < y)
q("Min_VisibilitykM", .95, lambda x,y: x > y)
q("Min_Humidity", .03, lambda x,y: x < y)
q("Min_Humidity", .95, lambda x,y: x > y)
q("Mean_Wind_SpeedKm_h", .95, lambda x,y: x > y)
q("Mean_TemperatureC", .03, lambda x,y: x < y)
q("Mean_TemperatureC", .95, lambda x,y: x > y)
q("Mean_VisibilityKm", .05, lambda x,y: x < y)
q("Mean_VisibilityKm", .95, lambda x,y: x > y)

# Normalization of Numerical Variables

In [ ]:
# take the numerical features
numeric_feats = sales.dtypes[sales.dtypes != "object"].index
# compute the skewness but only for non missing variables (we already imputed them but just in case ...)
skewed_feats = sales[numeric_feats].apply(lambda x: skew(x.dropna()))

skewness = pd.DataFrame({"Variable":skewed_feats.index, "Skewness":skewed_feats.data})
# select the variables with a skewness above a certain threshold
skewness = skewness.sort_values('Skewness', ascending=[0])
f, ax = plt.subplots(figsize=(8,6))
plt.xticks(rotation='90')
sns.barplot(x=skewness['Variable'], y=skewness['Skewness'])
plt.ylim(0,25)
plt.xlabel('Numerical Variables', fontsize=15)
plt.ylabel('Skewness', fontsize=15)
plt.title('', fontsize=15)

In [ ]:
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.drop(['IsHoliday','NumberOfCustomers', "NumberOfSales"])
sales[skewed_feats.index] = np.log1p(sales[skewed_feats.index])

# Correlation Analysis and Feature Selection

By analyzing the correlation, we see that the following variables do not add any additional information. 
Max_Dew_PointC, Min_Dew_PointC, Max_Sea_Level_PressurehPa, Mean_Sea_Level_PressurehPa, Max_Gust_SpeedKm_h
Finally, we drop 'NumberOfCustomers' because is not present in the submission dataset.

In [ ]:
sales=sales.drop(columns=['Max_Dew_PointC','Min_Dew_PointC','Max_Sea_Level_PressurehPa','Mean_Sea_Level_PressurehPa','Max_Gust_SpeedKm_h'])
sales.shape

Drop of the row where IsOpen==0

In [ ]:
sales=sales[sales['IsOpen']==1]
sales=sales.drop(columns=['IsOpen'])

### Convert Date to weekday label

In [ ]:
sales["Date"] = sales["Date"].apply(toDate)
sales["Day_Of_Week"] = sales["Date"].astype("datetime64").dt.weekday_name

### Convert Date to month label

In [ ]:
sales["Month"] = sales["Date"].astype("datetime64").dt.month

### Adding info about avgsales per month ecc.

In [ ]:
avgSalesForStoreIDForMonth = sales 
avgSalesForStoreIDForMonth = avgSalesForStoreIDForMonth.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].mean() 
avgSalesForStoreIDForMonth = avgSalesForStoreIDForMonth.rename(index=str, columns={"NumberOfSales": "AvgSalesForMonth"})

varSalesForStoreIDForMonth = sales 
varSalesForStoreIDForMonth = varSalesForStoreIDForMonth.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].var() 
varSalesForStoreIDForMonth = varSalesForStoreIDForMonth.rename(index=str, columns={"NumberOfSales": "VarSalesForMonth"})

avgCustomersForStoreIDForMonth = sales 
avgCustomersForStoreIDForMonth = avgCustomersForStoreIDForMonth.groupby(['StoreID','Month'], as_index=False)['NumberOfCustomers'].mean() 
avgCustomersForStoreIDForMonth = avgCustomersForStoreIDForMonth.rename(index=str, columns={"NumberOfCustomers": "AvgCustomersForMonth"})

varCustomersorStoreIDForMonth = sales 
varCustomersorStoreIDForMonth = varCustomersorStoreIDForMonth.groupby(['StoreID','Month'], as_index=False)['NumberOfCustomers'].var() 
varCustomersorStoreIDForMonth = varCustomersorStoreIDForMonth.rename(index=str, columns={"NumberOfCustomers": "VarCustomersForMonth"})

sales = sales.merge(avgSalesForStoreIDForMonth, left_on=['StoreID','Month'], right_on = ['StoreID','Month']) 
sales = sales.merge(varSalesForStoreIDForMonth, left_on=['StoreID','Month'], right_on = ['StoreID','Month'])
sales = sales.merge(avgCustomersForStoreIDForMonth, left_on=['StoreID','Month'], right_on = ['StoreID','Month']) 
sales = sales.merge(varCustomersorStoreIDForMonth, left_on=['StoreID','Month'], right_on = ['StoreID','Month'])

In [ ]:
sales.head(5)

# Dummify variables

In [ ]:
sales = pd.get_dummies(sales, columns=['StoreType','Events','AssortmentType', "Region", "Day_Of_Week","Month"])

# Train and Test Definition
Separating the last 2 months, and use those as a test set and comparing the total of the predicted values.

In [ ]:
start_train = date(2018, 1, 1)
train = sales[sales["Date"] - start_train < timedelta(0)]
test = sales[sales["Date"] - start_train > timedelta(days=1)]
train = train.drop(columns=["Date"])
test = test.drop(columns=["Date"])

Clusterization of regions which are similar or with too few samples to be fitted on their own.
Used t-sne to visualize clusters

In [ ]:
Region_labels = [['Region_0', 'Region_1', 'Region_4','Region_5','Region_7', 'Region_8', 'Region_10' ], ['Region_2'], ['Region_3'], ['Region_6', 'Region_9']]

In [ ]:
model = None
errors = []
for cluster in Region_labels:
    print("Cluster : " + str(cluster))
    model, errs = Fit(cluster,train,test, model)
    errors+=errs
from functools import reduce
print("Mean Error : ",reduce(lambda x, y: x+y, errors, 0)/len(errors))